In [33]:
import torch
from torch.utils.data import Dataset
import numpy as np
from PIL import Image
from torch.nn.utils.rnn import pad_sequence
import pdb
import h5py
from transformers import GPT2Tokenizer
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

class AttrDict(dict):
    __setattr__ = dict.__setitem__

    def __getattr__(self, attr):
        try:
            return self.__getitem__(attr)
        except KeyError:
            raise AttributeError("Attribute %r not found" % attr)

    def __getstate__(self):
        return self

    def __setstate__(self, d):
        self = d

def collate_batch(batch):

    behaviour_lengths = [item.behaviour_length for item in batch]

    images = [torch.tensor(item.observations) for item in batch]
    images = pad_sequence(images, batch_first=True, padding_value=0.0)

    actions = [torch.tensor(item.actions) for item in batch]
    actions = pad_sequence(actions, batch_first=True, padding_value=0.0)

    instructions = torch.stack([torch.tensor(item.instruction) for item in batch])

    gpt_tokens = torch.stack([item.gpt_tokens for item in batch])
    gpt_mask = torch.stack([item.gpt_mask for item in batch])

    gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

    # Decode gpt tokens into text
    text_list = []
    stop_words = ['close', 'move', 'next', 'push', 'put', 'slide', 'the', 'to', 'towards', '!', '\n']

    # Decode gpt tokens into text
    text_list = [gpt_tokenizer.decode(x).replace("!", "").split() for x in gpt_tokens]

    # Remove stop words
    stop_words = set(['close', 'move', 'next', 'push', 'put', 'slide', 'the', 'to', 'towards', '!', '\n'])
    text_list_clean = [[word for word in sublist if word not in stop_words] for sublist in text_list]

    # print(text_list_clean)
    # text_list_clean_array = np.array(text_list_clean)
    text_list_clean_array = np.array(text_list_clean, dtype=object)

    # print(text_list_clean_array)

    #similarity_matrix = (text_list_clean_array[:, None] == text_list_clean_array).all(axis=2)
    similarity_matrix = (text_list_clean_array[:, None] == text_list_clean_array).all()

    # print(similarity_matrix)
    
    # false_negative_mask = similarity_matrix

    # ??????????
    false_negative_mask = similarity_matrix  - np.eye(len(text_list_clean), dtype=float)
    false_negative_mask = np.logical_not(false_negative_mask).astype(float)
    false_negative_mask = np.where(false_negative_mask == 0.0, -np.inf, false_negative_mask)
    false_negative_mask = np.where(false_negative_mask == 0.0, -1e20, false_negative_mask)

    return AttrDict({'observations': images, 'actions': actions, 'instruction': instructions, 
                     'gpt_tokens': gpt_tokens, 'gpt_mask': gpt_mask, 'false_negative_mask': false_negative_mask,
                     'behaviour_lengths': behaviour_lengths})


class BehaviourDatasetCached(Dataset):
    def __init__(self, phase='train', data=None):

        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        self.gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

        self.all_data = data

        dataset_len = len(list(self.all_data.keys()))
        # dataset_len = 50000
        self.dataset_indices = np.arange(dataset_len)
        np.random.seed(0)
        np.random.shuffle(self.dataset_indices)

        if phase == 'train':
            self.dataset = self.dataset_indices[:int(0.8 * dataset_len)]
            print("Dataset length: ", len(self.dataset))

        elif phase == 'val':
            self.dataset = self.dataset_indices[int(0.8 * dataset_len):]
            print("Dataset length: ", len(self.dataset))

        self.max_seq_len = 25
        self.prefix_length = 10

        self.transform = transforms.Compose([
            transforms.Resize((180, 180)),
            transforms.ToTensor() 
        ])

    def pad_tokens(self, text_data):
        tokens = torch.tensor(self.gpt_tokenizer.encode(text_data), dtype=torch.int64)
        padding = self.max_seq_len - tokens.shape[0]
        if padding > 0:
            tokens = torch.cat((tokens, torch.zeros(padding, dtype=torch.int64) - 1))
        elif padding < 0:
            tokens = tokens[:self.max_seq_len]
        mask = tokens.ge(0)  # mask is zero where we out of sequence
        tokens[~mask] = 0
        mask = mask.float()
        mask = torch.cat((torch.ones(self.prefix_length), mask), dim=0)  # adding prefix mask
        return tokens, mask

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        index = self.dataset[idx]
        data = self.all_data[str(index)]
        observations = data["observations"][()]
        actions = data["actions"][()]
        instruction = data["clip_instruction_features"][()].squeeze(0)
        text_data = data["instruction"][()]
        behaviour_length = actions.shape[0]

        gpt_tokens, gpt_mask = self.pad_tokens(text_data.decode())
        
        return AttrDict({'observations': observations, 'actions': actions, 'instruction': instruction, 
                         'gpt_tokens': gpt_tokens, 'gpt_mask': gpt_mask, 'behaviour_length': behaviour_length})
    


dataset = h5py.File('/media/tim/E/datasets/lang_table/lang_table_test.h5', 'r')
train_dataset = BehaviourDatasetCached(phase='train', data=dataset)

train_dataloader = DataLoader(train_dataset, 
                                batch_size=64, 
                                shuffle=True, 
                                drop_last=True,
                                num_workers=6,
                                prefetch_factor=5,
                                collate_fn=collate_batch)


for batch in train_dataloader:
    print("observations: ", batch.observations.shape)
    print("instructions: ", batch.instruction.shape)
    print("actions: ", batch.actions.shape)
    print("gpt_tokens: ", batch.gpt_tokens.shape)
    print("gpt_mask: ", batch.gpt_mask.shape)
    print(batch.gpt_tokens[1])
    print(batch.gpt_mask[1])
    # print("behaviour_length: ", batch.behaviour_length)
    print("#############################################")
    break

Dataset length:  353780


2023-08-25 12:41:03.727006: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-25 12:41:03.748385: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-25 12:41:03.764118: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-25

observations:  torch.Size([64, 42, 2048])
instructions:  torch.Size([64, 512])
actions:  torch.Size([64, 42, 2])
gpt_tokens:  torch.Size([64, 25])
gpt_mask:  torch.Size([64, 35])
tensor([6649,  485,  262, 2266, 9197, 3371,  262, 4220, 3641,  220,  198,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
#############################################
